## Notes

This notebook only contains fitting and plotting code for the observed spectrum of HIP 65426 b as of January 2023, for the ExoGRAVITY collaboration. It does not include the code used to extract or flux calibrate this spectrum. The spectrum was extracted from one night of GRAVITY data for which there is a reliable on-axis observation, namely the Jan. 7th, 2021 observation of the system. The spectrum was extracted using the currently (as of Jan. 2023) ExoGRAVITY pipeline (see Appendix B in Nowak et al. 2020 for an outline of the data reduction steps), and flux calibrated using a BT-NextGen spectrum fit to archive photometry of the host, and scaled to match the SPHERE K1 photometry (about a factor of 0.5) of the companion. For more information on this, text for your paper, or for additional plots, contact W. Balmer :~)

This notebook assumes you have installed $\texttt{species}$, a toolkit for analyzing direct imaging observations. You can therefore change the model you fit to the observations (by default, the BT-Settl-Cifist model worked well when I was working on this target for the JWST ERS paper, but there are others listed on the species webpage), the number of live points, etc. and can save plots of the posterior of fit parameters, and the spectrum. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
# optional, set my personal preference for plots
# you can use this cell to define your plotting style instead
import seaborn as sb
plt.rcParams['font.family'] = 'monospace'   # Fonts
plt.rcParams['font.monospace'] = 'DejaVu Sans Mono'
sb.set_context("paper")

In [3]:
# initialize species
# this notebook will work, even if you have a clean installation of the package
# see https://species.readthedocs.io/ for more
import species
species.SpeciesInit()
database = species.Database()

/data/user/sblunt/miniconda3/envs/python3.8/lib/python3.8/site-packages/species/analysis/emission_line.py:18: UserWarning: UltraNest could not be imported. Perhaps because cython was not correctly compiled?
  warnings.warn(



ERROR:   Could not load MultiNest library "libmultinest.so"
ERROR:   You have to build it first,
ERROR:   and point the LD_LIBRARY_PATH environment variable to it!
ERROR:   manual: http://johannesbuchner.github.com/PyMultiNest/install.html


ERROR:   Could not load MultiNest library: libmultinest.so
ERROR:   You have to build MultiNest,
ERROR:   and point the LD_LIBRARY_PATH environment variable to it!
ERROR:   manual: http://johannesbuchner.github.com/PyMultiNest/install.html

problem: libmultinest.so: cannot open shared object file: No such file or directory

ERROR:   Could not load MultiNest library "libmultinest.so"
ERROR:   You have to build it first,
ERROR:   and point the LD_LIBRARY_PATH environment variable to it!
ERROR:   manual: http://johannesbuchner.github.com/PyMultiNest/install.html


ERROR:   Could not load MultiNest library: libmultinest.so
ERROR:   You have to build MultiNest,
ERROR:   and point the LD_LIBRARY_PATH environment variable to it!
ERROR:   manual: http://j

/data/user/sblunt/miniconda3/envs/python3.8/lib/python3.8/site-packages/species/analysis/fit_model.py:18: UserWarning: UltraNest could not be imported. Perhaps because cython was not correctly compiled?
  warnings.warn(
/data/user/sblunt/miniconda3/envs/python3.8/lib/python3.8/site-packages/species/analysis/fit_model.py:26: UserWarning: PyMultiNest could not be imported. Perhaps because MultiNest was not build and/or found at the LD_LIBRARY_PATH (Linux) or DYLD_LIBRARY_PATH (Mac)?
  warnings.warn(
/data/user/sblunt/miniconda3/envs/python3.8/lib/python3.8/site-packages/species/analysis/retrieval.py:26: UserWarning: PyMultiNest could not be imported. Perhaps because MultiNest was not build and/or found at the LD_LIBRARY_PATH (Linux) or DYLD_LIBRARY_PATH (Mac)?
  warnings.warn(


In [ ]:
# can determine your own filepath based on your directory structure
filepref = './data'

In [ ]:
# download and shape up the HIP 65426 b SPHERE spectrum
import urllib.request

urllib.request.urlretrieve('https://home.strw.leidenuniv.nl/~stolker/species/spectra/hip65426_sphere_yjh.dat', filepref+'hip65426_sphere_yjh.dat')

sphere_yjh = pd.read_csv(filepref+'hip65426_sphere_yjh.dat', delim_whitespace=True, comment='#', names=['l','f','fe'])
sphere_yjh = sphere_yjh[2:]
sphere_yjh.to_csv(filepref+'hip65426_sphere_yjh.dat', sep=' ', header=False, index=False)

In [ ]:
plx = (9.3031,0.0346)

In [ ]:
database.add_object('HIP 65426 b',
                    parallax=plx,
                    app_mag=None,
                    flux_density=None,
                    spectrum={
                            #   'GRAVITY': (filepref+'HIP65426B_GRAVITYK_spectrum_fluxcal_scaled.fits', filepref+'HIP65426B_GRAVITYK_spectrum_fluxcal_scaled.fits', 500.),
                              'SPHERE_IFU': (filepref+'hip65426_sphere_yjh.dat', None, 40.),
                             },
                    deredden=None)

In [ ]:
# add archival companion photometry
database.add_companion(name='HIP 65426 b')

In [ ]:
database.add_object('HIP 65426 b',
                    parallax=plx,
                    app_mag={#Carter et al. ERS vegamag
'JWST/NIRCAM.F250M':(17.163, 0.140),
'JWST/NIRCAM.F300M':(16.827, 0.117),
'JWST/NIRCAM.F356W':(16.067, 0.115),
'JWST/NIRCAM.F410M':(15.781, 0.120),
'JWST/NIRCAM.F444W':(15.701, 0.126),
'JWST/MIRI.F1140C':(15.357, 0.093),
'JWST/MIRI.F1550C':(15.197, 0.177)
                            },
                    spectrum=None,
                    deredden=None)

# optionally, can include to remove NACO photometry that disagrees with JWST
# database.delete_data('objects/HIP 65426 b/Paranal/NACO.Lp')
# database.delete_data('objects/HIP 65426 b/Paranal/NACO.NB405')
# database.delete_data('objects/HIP 65426 b/Paranal/NACO.Mp')


In [ ]:
model_choice = 'bt-settl-cifist'

In [ ]:
database.add_model(model=model_choice, teff_range=(1000., 1700.))

In [ ]:
inc_spec = True

inc_phot = [
'JWST/NIRCAM.F250M',
'JWST/NIRCAM.F300M',
'JWST/NIRCAM.F356W',
'JWST/NIRCAM.F410M',
'JWST/NIRCAM.F444W',
'JWST/MIRI.F1140C',
'JWST/MIRI.F1550C',
'Paranal/SPHERE.IRDIS_D_H23_2',
'Paranal/SPHERE.IRDIS_D_H23_3',
'Paranal/SPHERE.IRDIS_D_K12_1',
'Paranal/SPHERE.IRDIS_D_K12_2',
           ]

fit = species.FitModel(object_name='HIP 65426 b',
                       model=model_choice,
                       bounds={# 'teff': (1200., 1700.),
                               # 'radius': (0.5, 2.),
                               # 'SPHERE_IFU': ((0.5, 1.5), None),
                               # 'GRAVITY': ((0.5, 1.5), None),
                              },
                       inc_phot=inc_phot,
                       inc_spec=inc_spec,
                       fit_corr=['SPHERE_IFU'],
                       weights=None)


In [ ]:
tag='hip65426-plusjwst'

In [ ]:
fit.run_multinest(tag=tag,
                  n_live_points=300,
                  output='multinest/',
                  prior={'mass': (7.1, 1.1)} # prior from evol model, can include or disregard
                 )

In [ ]:
post = species.plot_posterior(tag=tag,
                       offset=(-0.3 , -0.3),
                       # title_fmt=['.0f', '.2f', '.2f', '.2f', '.2f', '.2f', '.3f', '.1f'],
                       inc_luminosity=True,
                       inc_mass=True,
                       output=None)

In [ ]:
samples = database.get_mcmc_spectra(tag=tag,
                                    random=30,
                                    wavel_range=None,
                                    spec_res=500.)

best = database.get_median_sample(tag=tag)

read_model = species.ReadModel(model=model_choice,
                               wavel_range=None)

modelbox = read_model.get_model(model_param=best,
                                spec_res=500.,
                                smooth=True)

objectbox = database.get_object(object_name='HIP 65426 b',
                                inc_phot=True,
                                inc_spec=True)

objectbox = species.update_spectra(objectbox=objectbox,
                                   model_param=best)

residuals = species.get_residuals(datatype='model',
                                  spectrum=model_choice,
                                  parameters=best,
                                  objectbox=objectbox,
                                  inc_phot=True,
                                  inc_spec=True)

synphot = species.multi_photometry(datatype='model',
                                   spectrum=model_choice,
                                   filters=objectbox.filters,
                                   parameters=best)

species.plot_spectrum(boxes=[samples, modelbox, objectbox, synphot],
                      filters=objectbox.filters,
                      residuals=residuals,
                      plot_kwargs=[{'ls': '-', 'lw': 0.2, 'color': 'gray'},
                                   {'ls': '-', 'lw': 1., 'color': 'black'},
                                   {'GRAVITY': {'marker': '.', 'ms': 5., 'mew': 0., 'color': 'seagreen', 'ls': 'none', 'alpha': 0.3, 'label': 'GRAVITY'},
                                    'SPHERE_IFU': {'marker': 's', 'ms': 5., 'mew': 0., 'color': 'cornflowerblue', 'ls': '', 'alpha': 1, 'label': 'SPHERE IFU'},
                                    'Paranal/NACO.Lp': {'marker': 's', 'ms': 5., 'color': 'tomato', 'ls': 'none', 'label':'NACO'},
                                    'Paranal/NACO.NB405': {'marker': 's', 'markersize': 5., 'color': 'tomato', 'ls': 'none'},
                                    'Paranal/NACO.Mp': {'marker': 's', 'markersize': 5., 'color': 'tomato', 'ls': 'none'},
                                    'JWST/NIRCAM.F250M': {'marker': 's', 'markersize': 5., 'color': 'xkcd:mustard', 'ls': 'none', 'label':'NIRCAM'},
                                    'JWST/NIRCAM.F300M': {'marker': 's', 'markersize': 5., 'color': 'xkcd:mustard', 'ls': 'none'},
                                    'JWST/NIRCAM.F356W': {'marker': 's', 'markersize': 5., 'color': 'xkcd:mustard', 'ls': 'none'},
                                    'JWST/NIRCAM.F410M': {'marker': 's', 'markersize': 5., 'color': 'xkcd:mustard', 'ls': 'none'},
                                    'JWST/NIRCAM.F444W': {'marker': 's', 'markersize': 5., 'color': 'xkcd:mustard', 'ls': 'none'},
                                    'JWST/MIRI.F1140C': {'marker': 's', 'markersize': 5., 'color': 'purple', 'ls': 'none', 'label':'MIRI'},
                                    'JWST/MIRI.F1550C': {'marker': 's', 'markersize': 5., 'color': 'purple', 'ls': 'none'}
                                   },
                                    None],
                      xlim=(0.8, 20.),
                      # ylim=(1.15e-20, 2e-16),
                      ylim_res=(-6., 6.),
                      scale=('log', 'linear'),
                      offset=(-0.4, -0.05),
                      legend=[{'loc': 'lower left', 'frameon': False, 'fontsize': 11.},
                              {'loc': 'upper right', 'frameon': False, 'fontsize': 12.}],
                      figsize=(12., 6.),
                      quantity='flux density',
                      output=filepref+'HIP65426b-SED.pdf'
                     )